In [1]:
# Code for analyzing NFL big data bowl data (probably with Celia!!!)

In [221]:
%reload_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import nflDataFunctions as nfldf

pd.set_option('display.max_rows', 50)

In [297]:
# Check if I got it right on the plays where the second closest receiver is almost the same distance as the closest
# ----- almost always, but not quite. In the one play I found where it isn't right, there's a pass arrived and a passCaught...
# ----- ----- leading into my next comment...

# Check what event string was used on those plays...
# ----- using pass_arrived with priority is a bit worse...

# Check if the abbreviated names have a 1-1 correspondence with the full names
# ----- it's not perfect because the abbreviated names are subject to variation and my string detection isn't perfect. most are good though...

In [7]:
games, players, plays, pffData = nfldf.loadData()
weekData = nfldf.loadWeek()

In [88]:
playWeek = nfldf.playWeek(games, plays)

In [294]:
# Code to identify target Receiver
itrOutputs = nfldf.identifyTargetReceiver(weekData[0],plays,playWeek,weekId=1)
receiverName, descName, receiverDistance, receiverNextClosestDistance, qbNflId = itrOutputs

In [296]:
fsDiff = receiverNextClosestDistance-receiverDistance
idxCloseSecond = np.where(fsDiff<2)[0]
for idx in idxCloseSecond:
    print(f"idx: {idx} Target receiver: {receiverName[idx]}, descName: {descName[idx]}, d/d:{receiverDistance[idx]}/{receiverNextClosestDistance[idx]}")

idx: 31 Target receiver: Mohamed Sanu, descName: M.Sanu, d/d:0.2009975124224206/1.1720921465482157
idx: 51 Target receiver: Jarvis Landry, descName: J.Landry, d/d:1.7376133056580827/2.741751265158822
idx: 93 Target receiver: T.Y. Hilton, descName: T.Hilton, d/d:0.282842712474621/1.2461540835707317
idx: 119 Target receiver: Tyler Boyd, descName: T.Boyd, d/d:1.211156472137268/2.1088859618291336
idx: 227 Target receiver: DeAndre Hopkins, descName: D.Hopkins, d/d:4.514797891378971/6.019842190622611
idx: 252 Target receiver: Ryan Griffin, descName: D.Hopkins, d/d:2.9735668817095737/4.49257164661845
idx: 362 Target receiver: Larry Fitzgerald, descName: L.Fitzgerald, d/d:4.86457603496954/5.106564402805479
idx: 390 Target receiver: David Johnson, descName: D.Johnson, d/d:0.27018512172212567/0.7117583859709761
idx: 433 Target receiver: Taylor Gabriel, descName: T.Gabriel, d/d:0.5126402247190421/1.1739250401963586
idx: 451 Target receiver: Anthony Miller, descName: A.Miller, d/d:0.29529646120466

11-L.Fitzgerald.
A.Abdullah
A.Armah
A.Auclair
A.Blue
A.Brown
A.Callaway
A.Carr
A.Collins
A.Cooper
A.Cross
A.Derby
A.Ekeler
A.Erickson
A.Firkser
A.Gates
A.Green
A.Green.
A.Holmes
A.Hooper
A.Humphries
A.Hurns
A.Janovich
A.Jeffery
A.Jones
A.Kamara
A.Lazard
A.Luck
A.Miller
A.Morris
A.Peterson
A.Roberts
A.Robinson
A.Seferian-Jenkins
A.Shaheen
A.Sherman
A.Tate
A.Thielen
A.Villanueva
A.Wilson
B.Bell
B.Bolden
B.Braunecker
B.Butler
B.Cooks
B.Cooks.
B.Cunningham
B.Ellington
B.Fowler
B.Hill
B.Jarwin
B.LaFell
B.Marshall
B.Parker
B.Perriman
B.Powell
B.Quick
B.Roethlisberger
B.Sowell
B.Watson
B.Wilson
B.Zylstra
C.Anderson
C.Artis-Payne
C.Batson
C.Beasley
C.Beasley.
C.Beebe
C.Brate
C.Carson
C.Clay
C.Clement
C.Coleman
C.Conley
C.Core
C.Daniel
C.Davis
C.Edmonds
C.Godwin
C.Grant
C.Ham
C.Herndon
C.Hogan
C.Hyde
C.Ivory
C.Kirk
C.Kupp
C.Latimer
C.Manhertz
C.McCaffrey
C.Meredith
C.Moore
C.Patterson
C.Peake
C.Phillips
C.Prosise
C.Ridley
C.Rogers
C.Samuel
C.Sutton
C.Thompson
C.Uzomah
C.Wentz
C.West
C.Williams
